# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [132]:
import sqlite3    # Python interface for SQLite databases

In [133]:
%%sql  
-- Clean all dummy/old tables
DROP TABLE IF EXISTS dummy;
DROP TABLE IF EXISTS wines;
DROP TABLE IF EXISTS production;
DROP TABLE IF EXISTS producers;
DROP TABLE IF EXISTS sales;
DROP TABLE IF EXISTS clients;
DROP TABLE IF EXISTS places;
DROP TABLE IF EXISTS producer;
DROP TABLE IF EXISTS client;
DROP TABLE IF EXISTS wine;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [134]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [135]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)


From this point we will use __%%sql__ magic

In [136]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [137]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [138]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

#### Just as a reminder, let's describe each normalization form again :

#### FIRST NORMAL FORM (1NF)
First normal form (1NF) sets the very basic rules for an organized database.
For a table to be in the First Normal Form, it should follow the following 4 rules:
It should only have single(atomic) valued attributes/columns.
Values stored in a column should be of the same domain.
All the columns in a table should have unique names.
And the order in which data is stored, does not matter.
 
#### SECOND NORMAL FORM (2NF)
Second normal form (2NF) further addresses the concept of removing duplicative data:
It should meet all the requirements of the first normal form.
We have to remove subsets of data that apply to multiple rows of a table and place them in separate tables. And finally, relationships between these new tables and their predecessors through the use of foreign keys have to be created.

#### THIRD NORMAL FORM (3NF)
Third normal form (3NF) goes one large step further:
It has to meet all the requirements of the second normal form.
Furthermore, we have to remove columns that are not dependent upon the primary key.

#### BOYCE-CODD NORMAL FORM (BCNF OR 3.5NF)
The Boyce-Codd Normal Form, also referred to as the "third and half (3.5) normal form," adds one more requirement:
It has to meet all the requirements of the third normal form.
Furthermore, every determinant must be a candidate key.

#### FOURTH NORMAL FORM (4NF)
Finally, fourth normal form (4NF) has one additional requirement:
It has to meet all the requirements of the third normal form.
Furthermore, a relation is in 4NF if it has no multi-valued dependencies.


## Master 1 has to be splitted in 3 tables : 

### Table 1 "wines" (Wine Informations) : 
|*Attribute*             | *Description*                  |
|:-----------------------|:-------------------------------|
| NV (**primary key**)   | Wine number                    |
| CRU                    | Vineyard or group of vineyards |
| DEGRE                  | Alcohol content                |
| MILL                   | Vintage year                   |
                  
NV ------> CRU, DEGRE, MILL

### Table 2 "producers" (Producer Informations) :
|*Attribute*             | *Description*                  |
|:-----------------------|:-------------------------------|
| NP (**primary key**)   | Producer number                |
| NOM                    | Producer's last name           |
| PRENOM                 | Producer's first name          |
| REGION                 | Production region              |

NP ------> NOM, PRENOM, REGION

### Table 3 "production" (Production Information) (i.e. the Junction Table) : 
|*Attribute*             | *Description*                  |
|:-----------------------|:-------------------------------|
| NV (**primary key**)   | Wine number                    |
| NP (**primary key**)   | Producer number                |
| QTE                    | Number of bottles harvested    |
 
NP, NV ------> QTE

#### Master 1 table that was in First Normal Form (1NF) is now converted into the Third Normal Form (3NF).

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

#### Master 2 can be normalized by reducing it in the following tables (Table 1 with thewine informations was already done in the previous question) where attributtes are :

### Table 1 "wines" (Wine Informations) : 
|*Attribute*             | *Description*                  |
|:-----------------------|:-------------------------------|
| NV (**primary key**)   | Wine number                    |
| CRU                    | Vineyard or group of vineyards |
| DEGRE                  | Alcohol content                |
| MILL                   | Vintage year                   |
 
NV ------> CRU, DEGRE, MILL
 
### Table 2 "clients" (Client Informations) : 
|*Attribute*             | *Description*                                            |
|:-----------------------|:---------------------------------------------------------|
| NB (**primary key**)   | Client (buveur) number                                   |
| NOM                    | Client's last name                                       |
| PRENOM                 | Client's first name                                      |
| TYPE                   | Type of client by volume of purchases                    |
| REGION                 | Administrative Region (different to production region)   |

NB ------> NOM, PRENOM, TYPE, REGION

### Table 3 "sales" (Sales Informations) (i.e. junction table) : 
|*Attribute*               | *Description*                  |
|:-------------------------|:-------------------------------|
| NB (**primary key**)     | Client (buveur) number         |
| NV (**primary key**)     | Wine number                    |
| DATES (**primary key**)  | Buying date                    |
| LIEU (**primary key**)   | Place where the wine was sold  |
| QTE                      | Number of bottles bought       |
                               
NB, NV, DATES, LIEU ------> DATES, LIEU, QTE

#### Master 2 table that was in First Normal Form (1NF) is now converted into the Third Normal Form (3NF). Note that we accounted for the fact that a wine can be bought in multiple locations and multiple times per day.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [139]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [140]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
dummy:
  0: DEGRE(NUM)


In [141]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [142]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [143]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

In [144]:
%%sql  
-- Clean all dummy/old tables
DROP TABLE IF EXISTS dummy;
DROP TABLE IF EXISTS wines;
DROP TABLE IF EXISTS production;
DROP TABLE IF EXISTS producers;
DROP TABLE IF EXISTS sales;
DROP TABLE IF EXISTS clients;
DROP TABLE IF EXISTS places;
DROP TABLE IF EXISTS producer;
DROP TABLE IF EXISTS client;
DROP TABLE IF EXISTS wine;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [145]:
%%sql DROP TABLE IF EXISTS wines;
CREATE TABLE wines AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;
SELECT *
FROM wines
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [146]:
%%sql DROP TABLE IF EXISTS producers;
CREATE TABLE producers AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL;
SELECT *
FROM producers
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne
18,Tramier,Jean,Bourgogne
19,Dupaquier,Roger,Bourgogne
20,Lamy,Jean,Bourgogne
21,Cornu,Edmond,Bourgogne
26,Violot,Gilbert,Bourgogne


In [147]:
%%sql DROP TABLE IF EXISTS production;
CREATE TABLE production AS
SELECT DISTINCT NP, NV, QTE
FROM MASTER1
WHERE NP IS NOT NULL AND NV IS NOT NULL;
SELECT *
FROM production
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NV,QTE
1,1,300
73,1,1
5,2,100
1,3,400
10,4,35
30,5,46
42,6,300
98,7,60
90,8,12
98,10,100


#### Exercise 1.4

Create the new tables from Master2:

In [148]:
%%sql DROP TABLE IF EXISTS sales;
CREATE TABLE sales AS
SELECT DISTINCT NB, NV, DATES, LIEU, QTE 
FROM MASTER2
WHERE NB IS NOT NULL AND NV IS NOT NULL AND DATES IS NOT NULL AND LIEU IS NOT NULL;
SELECT *
FROM sales
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NV,DATES,LIEU,QTE
2,1,1977-11-02,BORDEAUX,33
44,1,2015-10-16,PARIS,1
45,1,1983-12-31,RENNES,1
48,2,1983-12-25,LYON,2
7,3,1978-11-01,NICE,6
49,3,1987-12-24,MERCUREY,5
8,4,1982-11-05,TOULOUSE,12
44,4,2015-10-16,PARIS,6
50,4,1981-06-29,SENS,4
2,5,1984-11-04,MERCUREY,50


In [149]:
%%sql DROP TABLE IF EXISTS clients;
CREATE TABLE clients AS
SELECT DISTINCT NB, PRENOM, NOM, TYPE, REGION
FROM MASTER2
WHERE NB IS NOT NULL;
SELECT *
FROM clients
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,PRENOM,NOM,TYPE,REGION
11,Andre,Breton,petit,None
13,Roland,Barthes,moyen,None
16,Honore de,Balzac,moyen,None
18,Louis Ferdinand,Celine,gros,None
20,Francois-Rene de,Chateaubriand,moyen,None
21,Tristan,Corbiere,petit,None
23,Pierre,Corneille,petit,None
25,Rene,Char,petit,None
27,Alexandre,Dumas,gros,None
29,Alain,Fournier,petit,None


In [150]:
printSchema(conn)


MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
clients:
  0: NB(NUM)
  1: PRENOM(TEXT)
  2: NOM(TEXT)
  3: TYPE(TEXT)
  4: REGION(TEXT)
producers:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wines:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [151]:
%%sql SELECT DISTINCT TYPE 
FROM clients;

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [152]:
%%sql SELECT wines.CRU, producers.REGION
FROM wines
    JOIN production ON       wines.NV = production.NV
    JOIN producers  ON  production.NP = producers.NP
WHERE wines.CRU  = 'Pommard' OR wines.CRU = 'Brouilly';

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [153]:
%%sql SELECT  producers.REGION 
FROM wines
    JOIN production ON       wines.NV = production.NV
    JOIN producers  ON  production.NP = producers.NP
WHERE wines.CRU  = 'Pommard' OR wines.CRU = 'Brouilly' 
GROUP BY producers.REGION HAVING COUNT(wines.CRU) = 2 ;

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [154]:
%%sql SELECT  wines.CRU, wines.MILL, SUM(sales.QTE)
FROM wines
    JOIN sales ON       wines.NV = sales.NV 
GROUP BY wines.CRU,wines.MILL;

 * sqlite:///wine.db
Done.


CRU,MILL,SUM(sales.QTE)
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,520
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [155]:
%%sql SELECT  wines.NV, COUNT(producers.NP)
FROM wines
    JOIN production ON       wines.NV = production.NV
    JOIN producers  ON  production.NP = producers.NP 
    GROUP BY wines.NV
    HAVING COUNT(producers.NP)>3;

 * sqlite:///wine.db
Done.


NV,COUNT(producers.NP)
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [156]:
%%sql SELECT producers.NP,producers.NOM,producers.PRENOM,COUNT(production.NV)
    FROM producers LEFT JOIN production  
        ON  production.NP = producers.NP 
          WHERE production.NV IS NULL
            GROUP BY producers.NP ;
            

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM,COUNT(production.NV)
3,Six,Paul,0
6,Marmagne,Bernard,0
8,Lioger d'Harduy,Gabriel,0
16,Barbin,Bernard,0
17,Faiveley,Guy,0
18,Tramier,Jean,0
19,Dupaquier,Roger,0
20,Lamy,Jean,0
21,Cornu,Edmond,0
26,Violot,Gilbert,0


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [157]:
%%sql SELECT clients.NB,clients.NOM,clients.PRENOM
FROM clients
    JOIN sales  ON       clients.NB = sales.NB
    JOIN wines  ON         sales.NV = wines.NV 
    WHERE wines.MILL = '1980'
    GROUP BY clients.NB ;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [161]:
%%sql SELECT clients.NB,clients.NOM,clients.PRENOM,wines.MILL
FROM clients
JOIN sales  ON         clients.NB = sales.NB
JOIN wines  ON           sales.NV = wines.NV  
WHERE clients.NB NOT IN ( SELECT clients.NB FROM clients WHERE wines.MILL = '1980' )
GROUP BY clients.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,MILL
1,Aristote,None,1975
2,Artaud,Antonin,1976
3,Aron,Raymond,1981
4,Apollinaire,Guillaume,1968
5,Audiberti,Jacques,1973
6,Arrabal,Fernando,1975
7,Anouilh,Jean,1978
8,Aragon,Louis,1976
9,Ajar,Emile,1975
10,Andersen,Yann,1973


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [174]:
%%sql SELECT clients.NB,clients.NOM,clients.PRENOM,wines.MILL
FROM clients
JOIN sales  ON         clients.NB = sales.NB
JOIN wines  ON           sales.NV = wines.NV  
WHERE wines.MILL = '1980' AND clients.NB NOT IN ( 
    SELECT clients.NB FROM clients 
    JOIN sales  ON         clients.NB = sales.NB
    JOIN wines  ON           sales.NV = wines.NV  
    WHERE wines.MILL <> '1980' )
GROUP BY clients.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,MILL
44,Gide,Andre,1980
45,Giono,Jean,1980
50,Lautreamont,None,1980


#### Exercise 2.10

List all wines from 1980

In [169]:
%%sql SELECT  wines.NV, wines.CRU, wines.MILL, wines.DEGRE 
FROM wines
    WHERE wines.MILL='1980';

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [170]:
%%sql SELECT  wines.NV, wines.CRU, wines.MILL, wines.DEGRE
FROM wines
JOIN sales    ON           sales.NV = wines.NV 
WHERE wines.MILL='1980' AND sales.NB=2 ;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [177]:
%%sql SELECT clients.NB,clients.NOM,clients.PRENOM 
FROM clients
JOIN sales  ON         clients.NB = sales.NB
JOIN wines  ON           sales.NV = wines.NV  
WHERE wines.MILL='1980' 
GROUP BY clients.NB
HAVING COUNT(wines.NV) = ( 
    SELECT COUNT(wines.MILL) FROM wines 
    WHERE wines.MILL = '1980' ) ;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
